In [10]:
!python3.11 -m pip install lightning

In [11]:
from torch import nn

In [12]:
class MLP(nn.Module):
    def __init__(self, width, depth):
      super().__init__()
      self.layers = nn.ModuleList()
      self.layers.append(nn.Linear(10, width))
      self.layers.append(nn.LeakyReLU())

      self.layers.extend([nn.Linear(width, width), nn.LeakyReLU()] * depth)

      self.layers.append(nn.Linear(width, 1))

    def forward(self, x):
      result = x
      for layer in self.layers:
          result = layer(result)
      return result

In [13]:
from torch.utils.data import Dataset
import pandas as pd

class OtwayData(Dataset):
  def __init__(self):
    """Loads dataset from file"""
    df = pd.read_csv("cleaned_data.csv")
    self.y = df["Close"]
    self.X = df.drop("Close", axis=1)
    self.len = self.X.shape[0]

  def __getitem__(self, index):
    return self.X[index], self.y[index]
  
  def __len__(self):
    return self.len

In [14]:
import lightning as L
import torch

class OtwayModel(L.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = MLP(width=128, depth=4)

  def loss_function(predicted_y, true_y):
    loss_fn = nn.MSELoss()
    return loss_fn(predicted_y, true_y)

  def training_step(self, batch, batch_idx):
    features, true_y = batch
    predicted_y = self.model(features)
    loss = self.loss_function(predicted_y, true_y)
    return loss
  
  def validation_step(self, batch, batch_idx):
    features, true_y = batch
    predicted_y = self.model(features)
    loss = self.loss_function(predicted_y, true_y)
    self.log("val_loss", loss)
  
  def test_step(self, batch, batch_idx):
    features, true_y = batch
    predicted_y = self.model(features)
    loss = self.loss_function(predicted_y, true_y)
    self.log("test_loss", loss)
  
  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    return optimizer

In [15]:
from torch.utils.data import DataLoader, random_split

dataset = OtwayData()
train_set, test_set = random_split(dataset, [0.8, 0.2])
train_set, val_set = random_split(train_set, [0.8, 0.2])
train_loader = DataLoader(train_set)
val_loader = DataLoader(val_set)
test_loader = DataLoader(test_set)

In [16]:
model = OtwayModel()
trainer = L.Trainer()
trainer.fit(model, train_loader, val_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/lightning/pytorch/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.


MisconfigurationException: No `configure_optimizers()` method defined. Lightning `Trainer` expects as minimum a `training_step()`, `train_dataloader()` and `configure_optimizers()` to be defined.